### REGRESIÓN LINEAL EN PYTHON Y VARIABLES PROXY

##### Para esta clase utilizaremos como referencia la hipótesis y datos utilizados en el artículo: **"The Colonial Origins of Comparative Development: An Empirical Investigation"**

* Autores: Daron Acemoglu, Simon Johnson, James A. Robinson

* Fuente: The American Economic Review, Vol. 91, No. 5 (Dec., 2001), pp. 1369-1401 https://economics.mit.edu/files/4123

* La versión en español la pueden encontrar en  redalyc.org/pdf/419/41901302.pdf

#### PROPÓSITO DE ESTA CLASE
* Aplicar el concepto de variables instrumentales que ya revisamos de forma teórica.
* Utilizando un modelo de 2 etapas analizar la hipótesis de los autores: el desempeño económico observado puede ser atribuido a las diferencias institucionales.

* **Los autores del artículo proponen 3 premisas:**

* 1. Los diversos tipos de políticas de colonización crearon diferentes grupos de instituciones.  En un extremo, los europeos establecieron “Estados extractivos” , instituciones que  no proporcionaron mucha protección a la propiedad privada, ni establecieron un sistema de pesos y contrapesos contra la  expropiación del gobierno. 

* En el otro extremo, muchos europeos emigraron y se asentaron en diversas colonias, creando  “nuevas Europas”. Los colonizadores trataron de replicar las instituciones europeas, con gran énfasis en la propiedad privada y en el control del poder del gobierno.

* 2. La factibilidad de los asentamientos influyó en la estrategia de colonización. En lugares donde el ambiente insalubre no era favorable al asentamiento europeo, no resultaba posible crear “nuevas Europas”,y era más factible la formación del Estado extractivo.

* 3. El Estado colonial y las instituciones persistieron aun después de la independencia.


* **HIPÓTESIS:** las tasas de mortalidad (potenciales) de los colonizadores fueron el principal determinante de los asentamientos; los asentamientos fueron un determinante importante de las instituciones iniciales (en la práctica, las instituciones de 1900); y existe una fuerte correlación entre las instituciones iniciales y las instituciones actuales.

**¿Cómo medimos las diferencias institucionales y los resultados económicos?**

* Los resultados económicos de una economía son aproximados por el logaritmo del PIB per cápita en 1995, ajustados por la paridad del poder adquisitivo, ppp.

* Las diferencias institucionales son aproximadas mediante el  índice de protección contra la expropiación que reporta un  valor entre 0 y 10 para cada país y año,donde 0 corresponde a la menor protección contra la expropiación. Se utilizó  el valor promedio de cada país entre 1985 y 1995 ( “riesgo de expropiación” promedio 1985-95", este índice fue construido por el Grupo de Servicios de Riesgo Político https://www.prsgroup.com/).

* La principal contribución del artículo es el uso de las tasas de mortalidad como fuente de variación exógena en las diferencias institucionales.

* Dicha variación es necesaria para determinar si son las instituciones las que dan lugar a un mayor crecimiento económico, y no al revés.



### 1.- Importando las librerías necesarias
* Para esta clase necesitamos instalar linearmodels

In [ ]:
#!pip install pip
#!pip install linearmodels

In [ ]:
import numpy as np
import pandas as pd
#pd.core.common.is_list_like = pd.api.types.is_list_like
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from linearmodels.iv import IV2SLS
from linearmodels.iv.results import compare

#
import warnings
warnings.filterwarnings('ignore')

### 2.-Datos.
* Con la ayuda de la instrucción *read_stata* leeremos los datos, nuestro Dataframe contiene la siguiente información:
- shortnam - Abreviatura Nonmbre país.
- euro1900 -Restricciones al Ejecutivo en 1900
- excolony: variable dummy 1- excolonia -0 no excolonia
- logpgp95 logaritmo del pib per cápita en 1995 
- avexpr Protección promedio contra el riesgo de expropiación,1985-1995 
- cons1 Constraint on executive in first year of independence 
- cons90 Constraint on executive in 1900 
- democ00a ïndice de democracia 1900
- cons00a
- extmort4
- logem4 logaritmo de la tasa de mortalidad de los colonizadores para una muestra de 75 países,
- loghjypl
- baseco

In [ ]:
#
df1 = pd.read_stata('maketable1.dta')
df1.head()

### 3.- Explorando los datos de forma gráfica
* Con una gráfica de dispersión exploraremos la relación entre las variables **'avexpr' Protección promedio contra el riesgo de expropiación,1985-1995 y 'logpgp95' Logaritmo del pib per cápita, 1975 y 1995**.
* ¿Qué tipo de relación observas en los datos?

In [ ]:
# NOTA:
# 
print(plt.style.available)

In [ ]:
#
plt.style.use('seaborn-v0_8-white') #ggplot, 'seaborn-v0_8-white'
df1.plot(x ='avexpr', y ='logpgp95', kind = 'scatter')
plt.title('Gráfica 1: Gráfico de dispersión')
plt.show()

### 4.- Modelo OLS Bivariado

* Si un alto índice de  Protección promedio contra el riesgo de expropiación es una medida de calidad institucional, entonces **"mejores instituciones"  parecen estar positivamente correlacionadas con mejores desempeño económico (medido como un alto PIB per capita).**

* Para describir esta relación, podríamos generar un **modelo bivariado entre el PIB percápita como variable dependiente y el Índice de Protección  promedio contra el riesgo de expropiación  como variable explicativa.**

In [ ]:
# Generamos un subconjunto limpio de datos eliminando las observaciones con datos NA
df1_subset = df1.dropna(subset = ['logpgp95', 'avexpr'])

df1_subset.head()

In [ ]:
#
df1_subset = df1_subset[df1_subset['baseco'] == 1]

df1_subset.tail()

In [ ]:
#Establecemos los parámetros X y Y
X = df1_subset['avexpr'] 
y = df1_subset['logpgp95']

#Guardamos los valores de nuestras etiquetas 
labels = df1_subset['shortnam']

# Reemplazamos los marcadores con los nombres de las etiquetas
fig, ax = plt.subplots()
ax.scatter(X, y, marker = '')

for i, label in enumerate(labels):
    ax.annotate(label, (X.iloc[i], y.iloc[i]))

# Trazamos una línea de tendencia
ax.plot(np.unique(X),
         np.poly1d(np.polyfit(X, y, 1))(np.unique(X)),
         color = 'black')

ax.set_xlim([3.3,10.5])
ax.set_ylim([4,10.5])
ax.set_xlabel('Protección promedio contra el riesgo de expropiación 1985-95')
ax.set_ylabel('Log PIB per capita, PPP, 1995')
ax.set_title('Gráfica 2: OLS relación entre riesgo de expropiación e ingreso')

# Save the Figure
#plt.savefig("FIGURA.png", bbox_inches = 'tight')

plt.show()

* Para estimar los parámetros del modelo OLS, necesitamos añadir una columna con valor 1 

In [ ]:
#
df1['const'] = 1

df1.head()

* Ahora podemos construir el modelo OLS

In [ ]:
#
reg1 = sm.OLS( endog = df1['logpgp95'], exog = df1[['const', 'avexpr']], \
       missing = 'drop')

results = reg1.fit()

print(results.summary())

### Resultados:
**intercepto=4.63, B1=0.53,** la calidad institucional tiene un efecto positivo  en el desempeño económico.
**p-value:** El efecto de las instituciones sobre el PIB per cápita  es estadísticamente significativo
**R²=.611** al rededor de 61% de la variación en la variable LOG PIB per cápita es explicada por la variable Índice de  Protección promedio contra el riesgo de expropiación

### Utilizando nuestro modelo para predecir niveles de PIB per cápita

In [ ]:
#primero calculamos el promedio de avexpr

mean_expr = np.mean(df1_subset['avexpr'])

mean_expr

In [ ]:
results.predict(exog = [1, mean_expr] )

### Podemos obtener el valor predicho para cada valor 

In [ ]:
# Eliminamos los valores nulos de la base

df1_plot = df1.dropna( subset = ['logpgp95', 'avexpr'] )

# Graficamos los valores predichos

fig, ax = plt.subplots()
ax.scatter( df1_plot['avexpr'], 
            results.predict(), 
            alpha = 0.5,
            label = 'Predicción' )

# Graficamos los valores observados

ax.scatter( df1_plot['avexpr'], 
            df1_plot['logpgp95'], 
            alpha = 0.5, 
            label = 'Observado')

ax.legend()
ax.set_title( 'OLS valores predichos' )
ax.set_xlabel( 'avexpr' )
ax.set_ylabel( 'logpgp95' )

plt.show()

###  5.-Modelo Multivariado
* Hasta ahora, nuestro modelo sólo considera como variable explicativa el Índice de  Protección promedio contra el riesgo de expropiación, seguramente existen otras variables que tienen un efecto sobre el PIB per cápita.

* Los cálculos de los parámetros de nuestro modelo pueden estar afectados por lo que se conoce como **"sesgo de variable omitida"**, para solucionar este problema, extenderemos nuestro modelo para incluir otras variables.


In [ ]:
# cargamos los datos para nuestro modelo multivariado

df2 = pd.read_stata('maketable2.dta')

# Creamos nuestra constante
df2['const'] = 1

df2

In [ ]:
# Creamos una lista de variables que serán utilizadas en cada regresión 
# (vamos a generar 3 modelos) 
# Seleccionamos columnas mediante estas 3 listas:
X1 = ['const', 'avexpr']
X2 = ['const', 'avexpr', 'lat_abst']
X3 = ['const', 'avexpr', 'lat_abst', 'asia', 'africa', 'other']

# Estimaremos un modelo de regresión OLS por cada conjunto de variables.

reg1 = sm.OLS( df2['logpgp95'], df2[X1], missing = 'drop').fit()
reg2 = sm.OLS( df2['logpgp95'], df2[X2], missing = 'drop').fit()
reg3 = sm.OLS( df2['logpgp95'], df2[X3], missing = 'drop').fit() 

* Con la instrucción **summary_col** vamos a desplegar los resultados de los tres modelos en una sola tabla

In [ ]:
#
info_dict = { 'R-squared' : lambda x: f"{x.rsquared:.2f}",
              'No. observations' : lambda x: f"{int(x.nobs):d}" }

results_table = summary_col( results = [reg1, reg2, reg3] ,
                             float_format = '%0.2f',
                             stars = True,
                             model_names = [ 'Model 1',
                                             'Model 3',
                                             'Model 4'],
                             info_dict = info_dict,
                             regressor_order = [ 'const',
                                                 'avexpr',
                                                 'lat_abst',
                                                 'asia',
                                                 'africa' ])

results_table.add_title('Table 2 - OLS Regressions')

print(results_table)

### 6. Endogeneidad Modelo de Mínimos Cuadrados en dos etapas
* **Endogeneidad** puede surgir como resultado de un error de medición, autorregresión con autocorrelación de errores, simultaneidad y variables omitidas. Utilizando un modelo OLS de dos etapas revisaremos cómo podemos arreglar este problema.

* La relación que existe entre el Índice de protección  promedio contra el riesgo de expropiación ('avexpr') y el    Logaritmo del PIB per cápita, puede ser bidireccional. 
* Por ejemplo, es probable que los países más ricos puedan financiar o preferir mejores instituciones; o que las variables que afectan el ingreso también pueden estar correlacionadas con diferencias institucionales; también podría se plausible que  la construcción del índice de protección  promedio contra el riesgo de expropiación pudo sesgarse,  los analistas pueden estar predispuestos a ver que los países con mayores ingresos tengan mejores instituciones

### Instrumentos y Método de Variables Intrumentales en dos etapas

* Instrimentemos nuestro índice de protección a la democracia a través de una variable instrumental: la tasa de mortalidad de los primeros colonizadores.

* De esta forma utilizaremos el procedimiento de estimación de Mínimos Cuadros en Dos Etapas. Podemos utilizar el estimador de Variables Instrumentales para determinar (Segunda Etapa):

$$\hat{\boldsymbol{\beta}}^{IV} = (\hat{\mathbf{X}}' \hat{\mathbf{X}})^{-1} \hat{\mathbf{X}}' \mathbf{Y}$$

* Por otro lado, podemoos establecer el siguiente vector de innstrumentos:
\begin{equation*}
    \mathbf{z}_i = (1, x_{i1}, \ldots, x_{iK-1}, z_{i1}, \ldots, z_{iM})
\end{equation*}

* Contruyendo de forma simimar a otras matrices a $\mathbf{Z}$ apilado la información de cada uno de los individuos. De esta forma podremos constriuir $\hat{\mathbf{X}}$ mediante el uso de un estimador de MCO:
\begin{eqnarray*}
    \hat{\mathbf{X}} & = & \mathbf{Z} \hat{\boldsymbol{\gamma}} \\
    & = & \mathbf{Z} (\mathbf{Z}' \mathbf{Z})^{-1} \mathbf{Z}' \mathbf{X}
\end{eqnarray*}

* De lo anterior tendríamos que (Primera Etapa):
\begin{equation*}
    \hat{\mathbf{X}}' = \mathbf{X}' \mathbf{Z} (\mathbf{Z}' \mathbf{Z})^{-1} \mathbf{Z}'
\end{equation*}

* Sólo para poner en contexto, podemos platear el Método Generalizado de Momentos de la siguiente forma:


$$\hat{\boldsymbol{\beta}}^{GMM} = (\hat{\mathbf{X}}' \hat{\mathbf{W}} \hat{\mathbf{X}})^{-1} \hat{\mathbf{X}}'\hat{\mathbf{W}} \mathbf{Y}$$

Donde $\hat{\mathbf{W}}$ es una matriz definida positiva.

In [ ]:
# Regresamos a nuestro DF inicial: Borrado de NA's 
df1_subset2 = df1.dropna( subset = [ 'logem4', 'avexpr' ] )

X = df1_subset2['logem4']
y = df1_subset2['avexpr']

labels = df1_subset2['shortnam']

In [ ]:
plt.style.use('seaborn-v0_8-white')

# 
fig, ax = plt.subplots()
ax.scatter(X, y, marker = '')

for i, label in enumerate(labels):
    ax.annotate(label, (X.iloc[i], y.iloc[i]))

# Línea de tendencia
ax.plot( np.unique(X),
         np.poly1d(np.polyfit(X, y, 1))(np.unique(X)),
         color = 'darkblue' )

ax.set_xlim([1.8,8.4])
ax.set_ylim([3.3,10.4])
ax.set_xlabel('Logaritmo de la mortalidad de los colonizadores')
ax.set_ylabel('Riesgo promedio de expropiación 1985-95')
ax.set_title('Figura 3: Relación entre mortalidad de los colonizadores y el riesgo de expropiación', 
             size = 14)

#
plt.show()

### Primera Etapa: Para la primera etapa requerimos instrumentar el riesgo de expropiación

$$avexpr_i = \delta_0 + \delta_1 logem4_i + \nu_i$$

In [ ]:
# Import and select the data
df4 = pd.read_stata('maketable4.dta')
df4 = df4[df4['baseco'] == 1]
df4.head()

In [ ]:
# 
df4['const'] = 1

# Regresión
results_fs = sm.OLS(df4['avexpr'],
                    df4[['const', 'logem4']],
                    missing = 'drop').fit()

print(results_fs.summary())

### Segunda Etapa: En la segunda etapa estimamos la ecuación de interés

$$logpgp95_i = β_0 + β_1 \widehat{avexpr}_i + \varepsilon_i$$

In [ ]:
# Tomamos el valor predicho de la Primera Etapa:
df4['predicted_avexpr'] = results_fs.predict()

df4.head()

In [ ]:
# Estimamos la Segunda Etapa, mediante la estimación de la ecuación de interés
results_ss = sm.OLS(df4['logpgp95'],
                    df4[['const', 'predicted_avexpr']]).fit()

print(results_ss.summary())

### Estimación en una sola instrucción: IV2SLS

In [ ]:
# Sin estimación de ajuste de errores
iv = IV2SLS( dependent = df4['logpgp95'],
             exog = df4['const'],
             endog = df4['avexpr'],
             instruments = df4[[ 'logem4' ]]).fit()#.cov_type='unadjusted')

print(iv.summary)

### Estimación de varios modelos

In [ ]:
# 
iv_01 = IV2SLS( dependent = df4['logpgp95'],
                exog = df4['const'],
                endog = df4[['avexpr']],
                instruments = df4[[ 'logem4']]).fit()#.cov_type='unadjusted')

iv_02 = IV2SLS( dependent = df4['logpgp95'],
                exog = df4['const'],
                endog = df4[['avexpr', 'lat_abst']],
                instruments = df4[[ 'logem4', 'lat_abst']]).fit()#.cov_type='unadjusted')

iv_03 = IV2SLS( dependent = df4['logpgp95'],
                exog = df4['const'],
                endog = df4[['avexpr', 'lat_abst', 'africa', 'asia']],
                instruments = df4[[ 'logem4', 'lat_abst', 'africa', 'asia' ]]).fit()#.cov_type='unadjusted')
#

In [ ]:
#
result = compare({'Modelo 1': iv_01, 'Modelo 2': iv_02, 'Modelo 3': iv_03})

# Imprime el resultado
print(result)